In [3]:
import torch
import torch.nn as nn
from torch import Tensor
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from data import OpenSub, pad_batch, parse

In [5]:
params = {"batch_size":128, "train_path":"../data/t_given_s_train.txt", "test_path":"../data/t_given_s_test.txt", 
          "vocab_size":25000, "clip_thresh":5, "dict_path":"../Data/movie_25000", }